In [1]:
import numpy as np

In [6]:
def relu(Z):
  return np.maximum(0, Z)


relu(np.array([2, 3, -9]))
relu(np.array([[-3], [2]]))

array([2, 3, 0])

array([[0],
       [2]])

In [8]:
def relu_backward(dA, Z):
  return np.where(Z > 0, dA, 0)


relu_backward(
    np.array([[2, 3], [5.1, -2]]),
    np.array([[3, -4], [2, 1]]),
)

array([[ 2. ,  0. ],
       [ 5.1, -2. ]])

In [10]:
def sigmoid(Z):
  return 1 / (1 + np.exp(-Z))


sigmoid(np.array([[-5, 5, -2, 2]]))

array([[0.00669285, 0.99330715, 0.11920292, 0.88079708]])

In [13]:
def sigmoid_backward(dA, Z):
  s = sigmoid(Z)
  return dA * s * (1 - s)


sigmoid_backward(
    np.array([-2, 2, 5, -5]),
    np.array([-2, 2, 5, -5]),

)

array([-0.20998717,  0.20998717,  0.03324028, -0.03324028])

In [17]:
def bce_loss(A2, y):
  A2 = np.clip(A2, 1e-9, (1 - 1e-9))
  loss = -np.mean(y * np.log(A2) + (1-y) * (1-A2))
  return loss


bce_loss(
    np.array([0.2, 0.4, 0.7, 0.52]),
    np.array([0, 1, 1, 1]),
)

np.float64(0.28172303580488783)

In [21]:
def bce_loss_backward(A2, y):
  A2 = np.clip(A2, 1e-9, (1 - 1e-9))
  n = y.shape[0]
  return (-(y / A2) + (1 - y) / (1 - A2)) / n

In [31]:
A2 = np.array([0.2, 0.4, 0.7, 0.52])
y = np.array([0, 1, 1, 1])

In [60]:
bce_loss(
    A2=A2,
    y=y
)

lr = 0.01
dA2 = bce_loss_backward(A2=A2, y=y)
A2 -= lr * dA2

np.float64(0.1773032626574801)